In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

import warnings
warnings.filterwarnings("ignore")

# Langsmith Tracing
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
os.environ['LANGCHAIN_PROJECT'] = os.getenv('LANGCHAIN_PROJECT')
os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')

In [4]:
from langchain_groq import ChatGroq
from pprint import pprint

llm = ChatGroq(model='groq/compound-mini')
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001997D49E8D0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001997D52A870>, model_name='groq/compound-mini', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [5]:
from  langchain_core.output_parsers import StrOutputParser 
res = llm.invoke('Can you explain me about rules of f1 in 10 lines')
parser = StrOutputParser()
for line in parser.invoke(res).split("\n"):
    print(line)

Here are the rules of F1 in 10 lines:

1. Formula 1 (F1) is a single-seater racing championship with 20 cars and 10 teams.
2. Each team has 2 drivers competing in Grands Prix around the world.
3. Cars are designed and built by teams, with engines supplied by 4 manufacturers.
4. Qualifying sessions determine the starting grid for each Grand Prix.
5. The objective is to complete a set number of laps in the shortest time possible.
6. Drivers can make pit stops to change tires and refuel (not always necessary).
7. Overtaking is allowed, but drivers must stay on track and avoid contact.
8. Drivers are penalized for infractions like speeding in the pit lane or causing collisions.
9. Points are awarded to the top 10 finishers: 25, 18, 15, 12, 10, 8, 6, 4, 2, 1.
10. The driver and team with the most points at the end of the season are crowned World Champions.


### HumanMessages & System Messages

In [6]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
                SystemMessage(content="Translate the user message to French"),
                HumanMessage(content="You look beautiful tonight!")
]

response = llm.invoke(messages)
response.content

"You look beautiful tonight translates to:\n\nTu as l'air magnifique ce soir!\n\nor, in a more formal tone:\n\nVous êtes magnifique ce soir!"

In [7]:
from langchain_core.output_parsers import StrOutputParser

str_parser = StrOutputParser()
str_parser.invoke(response)

"You look beautiful tonight translates to:\n\nTu as l'air magnifique ce soir!\n\nor, in a more formal tone:\n\nVous êtes magnifique ce soir!"

In [8]:
# Using LCEL For Chaining

chain1 = llm|str_parser
chain1.invoke(messages)

'Tu es belle ce soir!'

In [9]:
from langchain_core.prompts import ChatPromptTemplate

gen_prompt  = "Translate the following into {language}"
prompt = ChatPromptTemplate.from_messages([
                ('system',gen_prompt),
                ("user", "{text}")
])

prompt.invoke({"language":"French","text":"Rahul is a really amazing person"})

ChatPromptValue(messages=[SystemMessage(content='Translate the following into French', additional_kwargs={}, response_metadata={}), HumanMessage(content='Rahul is a really amazing person', additional_kwargs={}, response_metadata={})])

In [10]:
chain2  = prompt|llm|str_parser
chain2.invoke({"language":"French","text":"Rahul is a really amazing person"})

'Rahul est une personne vraiment incroyable.'

In [11]:
# Simple Demonstration of how the History is stored 
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

llm.invoke([
    HumanMessage("Hi Im Rahul, Im a great AI Engineer"),
    AIMessage("Hello Rahul! Nice to meet you! It's great to hear that you're an AI engineer. What kind of AI projects have you been working on lately"),
    HumanMessage("Hey whats my name and what do I Do?")
])

AIMessage(content="Your name is Rahul, and you mentioned you're a great AI Engineer!", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 439, 'total_tokens': 471, 'completion_time': 0.058586, 'prompt_time': 0.069088, 'queue_time': 0.100453, 'total_time': 0.127674}, 'model_name': 'groq/compound-mini', 'system_fingerprint': None, 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--7f891b6b-d3b1-4197-a997-f54e69723a6f-0', usage_metadata={'input_tokens': 439, 'output_tokens': 32, 'total_tokens': 471})

### Chat Message History

In [12]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import HumanMessage
from langchain_core.output_parsers import StrOutputParser

store = {}

def get_session_history(session_id :str)->BaseChatMessageHistory: # This is the base message history
    if session_id not in store:
        store[session_id] = ChatMessageHistory() # This is the active session history
    return store[session_id]

with_chat_message_history = RunnableWithMessageHistory(llm,get_session_history)

config = {"configurable":{"session_id":"chat1"}}

response = with_chat_message_history.invoke([HumanMessage(content="Hi My Name is Rahul")],
                                 config=config)

op_parse = StrOutputParser()
op_parse.invoke(response)

"Hello Rahul! It's nice to meet you. Is there something I can help you with or would you like to chat?"